# 03b_cnn_train — 以 global/local 視圖訓練 1D‑CNN
先用合成資料 smoke test；接上真實資料時，替換為 BLS/TLS 產生之 period/t0/duration 與 (time,flux)。

In [ ]:
import os, json, numpy as np, torch
from torch.utils.data import random_split, DataLoader
from app.models.cnn1d import make_model
from app.data.fold import Item, LightCurveViewsDataset
from app.trainers.cnn1d_trainer import train
from app.calibration.calibrate import run_and_save
device = 'cuda' if torch.cuda.is_available() else 'cpu'; device

In [ ]:
def synth_curve(n=4096, period=2.5, depth=0.0015, duration=0.08, t0=0.5, noise=5e-4, seed=0):
    rng = np.random.default_rng(seed); t = np.linspace(0, period*6, n); y = np.ones_like(t)
    ph = ((t - t0)/period)%1.0; in_tr = (ph < duration/period); y[in_tr] -= depth
    y += rng.normal(0, noise, size=y.shape); return t, y
items=[]
for i in range(300):
    lbl=1 if i%2==0 else 0
    if lbl: t,y=synth_curve(seed=i)
    else:   t=np.linspace(0, 15, 4096); y=1+np.random.default_rng(i).normal(0,6e-4,size=t.shape)
    items.append(Item(time=t, flux=y, period=2.5, t0=0.5, duration=0.08, label=lbl))
ds=LightCurveViewsDataset(items); n_total=len(ds); n_train=int(n_total*0.8); n_val=n_total-n_train
train_ds, val_ds = random_split(ds,[n_train,n_val])
model = make_model(); metrics = train(model, train_ds, val_ds, device=device, batch_size=64, lr=1e-3, max_epochs=8, patience=3, workdir='.')
metrics

In [ ]:
from torch.utils.data import DataLoader
from app.models.cnn1d import make_model
mdl = make_model(); mdl.load_state_dict(torch.load('artifacts/cnn1d.pt', map_location=device)); mdl.to(device); mdl.eval()
val_loader = DataLoader(val_ds, batch_size=64)
probs=[]; ys=[]; import numpy as np, torch
with torch.no_grad():
    for G,L,Y in val_loader:
        G=torch.tensor(G,dtype=torch.float32,device=device); L=torch.tensor(L,dtype=torch.float32,device=device)
        logits=mdl(G,L).squeeze(1); probs.append(torch.sigmoid(logits).cpu().numpy()); ys.append(Y)
probs=np.concatenate(probs); ys=np.concatenate(ys)
from app.calibration.calibrate import run_and_save
cal_info = run_and_save(ys, probs, out_dir='artifacts', method='isotonic'); cal_info